In [1]:
import duckdb
import pandas as pd
from mosaic_widget import MosaicWidget

In [2]:
weather = pd.read_csv("../../data/seattle-weather.csv", parse_dates=['date'])
weather

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,rain
1457,2015-12-28,1.5,5.0,1.7,1.3,rain
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


In [3]:
# for now add the table to duckdb since otherwise the table_info pragam doesn't work
con = duckdb.connect()
con.execute("CREATE TABLE weather AS SELECT * FROM weather")

In [4]:
spec = {
  "params": {
    "click": {
      "select": "single"
    },
    "domain": ["sun", "fog", "drizzle", "rain", "snow"],
    "colors": ["#e7ba52", "#a7a7a7", "#aec7e8", "#1f77b4", "#9467bd"]
  },
  "vconcat": [
    {
      "hconcat": [
        {
          "plot": [
            {
              "mark": "dot",
              "data": {
                "from": "weather",
                "filterBy": "$click"
              },
              "x": {
                "dateMonthDay": "date"
              },
              "y": "temp_max",
              "fill": "weather",
              "r": "precipitation",
              "opacity": 0.7
            },
            {
              "select": "intervalX",
              "as": "$range"
            },
            {
              "select": "highlight",
              "by": "$range",
              "fill": "#eee"
            },
            {
              "legend": "color",
              "as": "$click",
              "columns": 1
            }
          ],
          "domainXY": "Fixed",
          "tickFormatX": "%b",
          "domainColor": "$domain",
          "rangeColor": "$colors",
          "domainR": "Fixed",
          "rangeR": [2, 10],
          "width": 800
        }
      ]
    },
    {
      "plot": [
        {
          "mark": "barX",
          "data": {
            "from": "weather"
          },
          "x": {
            "count": None
          },
          "y": "weather",
          "fill": "#f5f5f5"
        },
        {
          "mark": "barX",
          "data": {
            "from": "weather",
            "filterBy": "$range"
          },
          "x": {
            "count": None
          },
          "y": "weather",
          "fill": "weather"
        },
        {
          "select": "selectY",
          "as": "$click"
        },
        {
          "select": "highlight",
          "by": "$click"
        }
      ],
      "domainX": "Fixed",
      "domainY": "$domain",
      "labelY": None,
      "domainColor": "$domain",
      "rangeColor": "$colors",
      "width": 800
    }
  ]
}

In [5]:
mosaic = MosaicWidget(spec, con)
mosaic

MosaicWidget(spec={'params': {'click': {'select': 'single'}, 'domain': ['sun', 'fog', 'drizzle', 'rain', 'snow…